In [1]:
import pandas as pd
import numpy as np
import os
import config

In [2]:
!ls $config.DATA_DIR

Emoji_Sentiment_Data_v1.0.csv
training.csv
validation.csv


In [3]:
sentiment140 = pd.read_csv(os.path.join(config.DATA_DIR, 'training.csv'), encoding = 'ISO-8859-1', header = None)
df = sentiment140.iloc[:,[0,5]]
df = df.rename(columns = {0 : 'label', 5 : 'text'})
df['label'] = (df.label != 0).astype(np.int32)

In [5]:
emoji_stats = pd.read_csv(os.path.join(config.DATA_DIR, 'Emoji_Sentiment_Data_v1.0.csv'))

TOTAL_TWEETS = 1643735
p_pos = 614003/TOTAL_TWEETS
p_neg = 422457/TOTAL_TWEETS

emoji_stats['p_emoji'] = emoji_stats['Occurrences']/TOTAL_TWEETS

emoji_stats['p_pos'] = emoji_stats['Positive'] / emoji_stats['Occurrences']
emoji_stats['p_neg'] = emoji_stats['Negative'] / emoji_stats['Occurrences']

emoji_stats['p_e|pos'] = emoji_stats['p_pos'] * emoji_stats['p_emoji'] / p_pos
emoji_stats['p_e|neg'] = emoji_stats['p_neg'] * emoji_stats['p_emoji'] / p_neg

In [6]:
emojified_df = df.copy()

for i, row in enumerate(emoji_stats.iterrows()):
    row = row[1]
    emojified_df['rand'] = np.random.rand(len(emojified_df))
    mask = emojified_df.label * row['p_e|pos'] + (1 - emojified_df.label) * row['p_e|neg'] >= emojified_df.rand
    emojified_df.loc[mask,'text'] += ' ' + row.Emoji
    print('\rStep: ', i, end = '')
print('\nDone')

Step:  968 477
Done


In [7]:
emojified_df['text'] = emojified_df['text'].str.strip()

In [8]:
msk = np.random.rand(len(emojified_df)) < 0.99
train = emojified_df[msk]
test = emojified_df[~msk]

In [9]:
train.to_csv(os.path.join(config.DATA_DIR, 'emojified_train.csv'), index = False)
test.to_csv(os.path.join(config.DATA_DIR, 'emojified_test.csv'), index = False)